In [47]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import re
from keras.preprocessing.text import Tokenizer

In [2]:
#(x_train, y_train), (x_test, y_test) = imdb.load_data()

In [48]:
dataset = pd.read_csv(r'movie_reviews.csv')

print(dataset.head())

reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [49]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

clean_docs = []

for doc in reviews:
    doc = strip_html_tags(doc)
    doc = doc.lower().strip()
    doc = re.sub('[^a-zA-z0-9\s]','',doc)
    clean_docs.append(doc)

In [5]:
tokenizer = Tokenizer(num_words=10000, split=' ')
tokenizer.fit_on_texts(clean_docs)

In [6]:
tokenizer.word_index

{'kuttram': 88006,
 'payola': 56325,
 'viscousif': 161324,
 'sheso': 128104,
 'vsexy': 206912,
 'michalka': 48912,
 'intended': 1331,
 'rally': 12221,
 'coprology': 92641,
 'businesses': 17527,
 'caveshokey': 200779,
 'pieomys': 140773,
 'wallah': 44909,
 'beginnig': 154141,
 'onceto': 200875,
 'scenography': 59031,
 'storynow': 61116,
 'khoo': 136733,
 'peopleprudes': 122060,
 'dangerfieldi': 150247,
 'abbad': 162652,
 'pointcounterpoint': 153619,
 'houndside': 183681,
 'aarika': 73428,
 'ushpizin': 167708,
 'filmcontrary': 151610,
 'goodwin': 42771,
 'allthis': 15806,
 'monstersontherampage': 160875,
 'newell': 36639,
 'frequencies': 79149,
 'batjacproduction': 107750,
 'iswas': 21911,
 'bloodtaking': 129314,
 'whammo': 128803,
 'failsthen': 100142,
 'directiongrade': 144701,
 'rahims': 214038,
 'ramzi': 71792,
 'momentsfor': 111870,
 'pavloskys': 208507,
 'singlemalt': 183751,
 'zanzibar': 120214,
 'httpowlypttp': 142080,
 'spyl': 94961,
 'flatulence': 24471,
 '30th': 21731,
 'sidne

In [7]:
X = tokenizer.texts_to_sequences(clean_docs)

In [8]:
X = sequence.pad_sequences(X)

In [9]:
sentiment_ohe = np.array(pd.get_dummies(sentiments))
sentiment_ohe

array([[0, 1],
       [0, 1],
       [0, 1],
       ..., 
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [10]:
train_X = X[:35000]
train_y = sentiment_ohe[:35000]
test_X = X[35000:]
test_y = sentiment_ohe[35000:]

train_X.shape, train_y.shape, test_X.shape, test_y.shape

((35000, 2087), (35000, 2), (15000, 2087), (15000, 2))

In [19]:
embed_dim = 128
lstm_out = 64

model = Sequential()
model.add(Embedding(10000, embed_dim, input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2087, 128)         1280000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2087, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 1,329,538
Trainable params: 1,329,538
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
batch_size = 100
model.fit(train_X, train_y, epochs = 5, batch_size=batch_size, verbose=1)

Epoch 1/5
35000/35000 [==============================] - 5115s - loss: 0.4314 - acc: 0.8086   
Epoch 2/5
35000/35000 [==============================] - 5328s - loss: 0.3293 - acc: 0.8661   
Epoch 3/5
35000/35000 [==============================] - 5458s - loss: 0.2708 - acc: 0.8923   
Epoch 4/5
35000/35000 [==============================] - 5855s - loss: 0.2266 - acc: 0.9128   
Epoch 5/5
35000/35000 [==============================] - 5681s - loss: 0.1936 - acc: 0.9257   


In [23]:
r = model.predict_classes(test_X)

15000/15000 [==============================] - 503s   


In [29]:
a = np.array(pd.DataFrame(test_y).idxmax(axis=1))
a

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [43]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [42]:
print(classification_report(a, r, target_names=['Negative', 'Positive']))

             precision    recall  f1-score   support

   Negative       0.86      0.90      0.88      7490
   Positive       0.89      0.85      0.87      7510

avg / total       0.87      0.87      0.87     15000



In [45]:
confusion_matrix(a, r)

array([[6717,  773],
       [1135, 6375]])

In [46]:
reviews[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa